<h1>Invoking SageMaker Model EndPoints From Client Outside of AWS</h1>

In [1]:
# Train the best performing model. With Hourly Features and Log of Count
# Kaggle Score: 0.42658
import numpy as np
import pandas as pd
import os

# Define IAM role
import boto3
import re

import sagemaker

In [8]:
# use a different profile
boto_session = boto3.Session(profile_name='dev', region_name='us-west-2')

In [9]:
sess = sagemaker.Session(boto_session=boto_session)

In [10]:
#endpoint_name = 'xgboost-biketrain-v1'
endpoint_name = 'sagemaker-xgboost-2020-07-06-12-05-58-891'
predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name,
                                                 sagemaker_session=sess)

In [11]:
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [12]:
def run_predictions(arr_input, batch_size):
    predictions = []
    
    if type(arr_input) != np.ndarray:
        arr_input = np.array(arr_input)
        
    for arr in np.array_split(arr_input,batch_size):
        if arr.size > 0: 
            print("Shape:{0}".format(arr.shape))
            result = predictor.predict(arr)
            result = result.decode("utf-8")
            result = result.split(',')
            predictions += [np.expm1(float(r)) for r in result]
    
    return predictions

In [13]:
df_test = pd.read_csv('bike_test.csv')

In [14]:
df_test.head()

,2011-01-20 00:00:00,1,0,1.1,1.2,10.66,11.365,56,26.0027,2011,1.3,20,3,0.1
0,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
1,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
2,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
3,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4
4,2011-01-20 05:00:00,1,0,1,1,9.84,11.365,60,15.0013,2011,1,20,3,5


In [15]:
tmp_df = df_test.as_matrix(columns=['season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'dayofweek','hour'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


In [16]:
tmp_df.shape

(6492, 13)

In [17]:
predicted_count = run_predictions(arr_input=tmp_df, batch_size=10)

Shape:(650, 13)
Shape:(650, 13)
Shape:(649, 13)
Shape:(649, 13)
Shape:(649, 13)
Shape:(649, 13)
Shape:(649, 13)
Shape:(649, 13)
Shape:(649, 13)
Shape:(649, 13)


In [18]:
predicted_count[:5]

[15.102146701062873,
 15.102146701062873,
 15.102146701062873,
 15.102146701062873,
 15.102146701062873]

In [19]:
df_test['count'] = predicted_count

In [20]:
df_test.head()

,2011-01-20 00:00:00,1,0,1.1,1.2,10.66,11.365,56,26.0027,2011,1.3,20,3,0.1,count
0,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1,15.102147
1,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2,15.102147
2,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3,15.102147
3,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4,15.102147
4,2011-01-20 05:00:00,1,0,1,1,9.84,11.365,60,15.0013,2011,1,20,3,5,15.102147


In [21]:
df_test['count'].describe()

count    6.492000e+03
mean     1.510215e+01
std      1.147615e-12
min      1.510215e+01
25%      1.510215e+01
50%      1.510215e+01
75%      1.510215e+01
max      1.510215e+01
Name: count, dtype: float64